In [2]:
!pip install pyodbc

  Obtaining dependency information for pyodbc from https://files.pythonhosted.org/packages/06/50/06feaa1f60556a69560e00858d436df24eec47b25bfe69ea455639dc4745/pyodbc-5.0.1-cp38-cp38-win_amd64.whl.metadata
   ---------------------------------------- 0.0/68.8 kB ? eta -:--:--
   ---------------------------------------- 68.8/68.8 kB 3.7 MB/s eta 0:00:00


In [1]:
import pyodbc
import json
import random

# Replace 'your_connection_string' with your actual connection string
connection_string = r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=C:\Users\noamo\Documents\Summarticle_database.accdb;' #Change this path

def save_paper(json_data):
    connection = pyodbc.connect(connection_string)
    cursor = connection.cursor()

    paper_title = json_data.get("paper", {}).get("title")
    
    # Check if the paper title is not already in the Paper table
    select_query = "SELECT * FROM papers WHERE title = ?"
    cursor.execute(select_query, (paper_title,))
    
    if not cursor.fetchone():
        # Paper title is not in the Paper table, save the paper
        insert_query = "INSERT INTO papers (title, authors, DOI, keywords) VALUES (?, ?, ?, ?)"
        cursor.execute(insert_query, (
            paper_title,
            json_data.get("paper", {}).get("authors"),
            json_data.get("paper", {}).get("DOI"),
            ', '.join(json_data.get("paper", {}).get("keywords", []))
        ))
        
        connection.commit()

    cursor.close()
    connection.close()

def save_summary(json_data):
    connection = pyodbc.connect(connection_string)
    cursor = connection.cursor()

    paper_title = json_data.get("summary", {}).get("title")
    
    # Check if the paper title is not already in the Summary table
    select_query = "SELECT * FROM summary WHERE title = ?"
    cursor.execute(select_query, (paper_title,))
    
    if not cursor.fetchone():
        # Paper title is not in the Summary table, save the summary
        insert_query = "INSERT INTO summary (title, summary_en, rating_en) VALUES (?, ?, ?)"
        cursor.execute(insert_query, (
            paper_title,
            json_data.get("summary", {}).get("summary_en"),
            json_data.get("summary", {}).get("rating_en", 0)
        ))
        
        connection.commit()
    else:
        # Paper title is already in the Summary table, save summary in a new row
        insert_query = "INSERT INTO summary (title, summary_en, rating_en) VALUES (?, ?, ?)"
        cursor.execute(insert_query, (
            paper_title,
            json_data.get("summary", {}).get("summary_en"),
            json_data.get("summary", {}).get("rating_en", 0)
        ))

        connection.commit()

    cursor.close()
    connection.close()

def read_summary(json_data):
    connection = pyodbc.connect(connection_string)
    cursor = connection.cursor()

    paper_title = json_data.get("title")
    language = json_data.get("language", "en")

    # Check whether JSON GET request is from summary_en or summary_nl
    if language == "en":
        select_query = "SELECT summary_en, rating_en FROM summary WHERE title = ? AND summary_en IS NOT NULL"
    elif language == "nl":
        select_query = "SELECT summary_nl, rating_nl FROM summary WHERE title = ? AND summary_nl IS NOT NULL"
    else:
        return None  # Invalid language
    
    cursor.execute(select_query, (paper_title,))
    result = cursor.fetchone()

    if result:
        # If there is a summary in the requested language, return it based on the rating
        summary, rating = result
        return summary if random.random() < rating / 10.0 else None  # Weighted random based on rating
    else:
        return None  # Create a new summary (you already have code for this)

    cursor.close()
    connection.close()

def read_keyword(json_data):
    connection = pyodbc.connect(connection_string)
    cursor = connection.cursor()

    keyword = json_data.get("keyword")

    # Get keyword from JSON request
    select_query = "SELECT title FROM papers WHERE keywords LIKE ?"
    cursor.execute(select_query, ('%' + keyword + '%',))
    result = cursor.fetchall()

    if result:
        # If there are papers with the specified keyword, return the titles
        return [row[0] for row in result]
    else:
        return None

    cursor.close()
    connection.close()

def update_rating(json_data):
    connection = pyodbc.connect(connection_string)
    cursor = connection.cursor()

    paper_title = json_data.get("title")
    rating_type = json_data.get("rating_type")

    # If JSON-rating == thumbs-up, corresponding rating in table += 1
    # If JSON-rating == thumbs-down, corresponding rating in table -= 1
    if rating_type == "thumbs-up":
        update_query = "UPDATE summary SET rating_en = CASE WHEN rating_en + 1 > 10 THEN 10 ELSE rating_en + 1 END WHERE title = ?"
    elif rating_type == "thumbs-down":
        update_query = "UPDATE summary SET rating_en = CASE WHEN rating_en - 1 < 1 THEN 1 ELSE rating_en - 1 END WHERE title = ?"
    else:
        return None  # Invalid rating type

    cursor.execute(update_query, (paper_title,))
    connection.commit()

    cursor.close()
    connection.close()

def delete_summary(json_data):
    connection = pyodbc.connect(connection_string)
    cursor = connection.cursor()

    paper_title = json_data.get("title")

    # If summary_rating is lower than 3, delete from Summary table
    delete_query = "DELETE FROM summary WHERE title = ? AND rating_en < 3"
    cursor.execute(delete_query, (paper_title,))
    connection.commit()

    cursor.close()
    connection.close()
